In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [98]:
mnist = input_data.read_data_sets('./input_data', one_hot=True, validation_size=500)

BATCH_SIZE = 100
batchSamples, batchLabels = mnist.train.next_batch(BATCH_SIZE)
batchSamples = batchSamples.reshape((BATCH_SIZE, 28, 28, 1))
batchSamples = np.pad(batchSamples, ((0,0),(2,2),(2,2),(0,0)), 'constant', constant_values=0)

Extracting ./input_data\train-images-idx3-ubyte.gz
Extracting ./input_data\train-labels-idx1-ubyte.gz
Extracting ./input_data\t10k-images-idx3-ubyte.gz
Extracting ./input_data\t10k-labels-idx1-ubyte.gz


In [100]:
inTensor = batchSamples
assert inTensor is batchSamples

In [101]:
"""
第一层为卷积层, 卷积层_1的参数  
input:  filter: 
        size: 5*5;  number: 6  channel:  1  
output: feature map: 
        size 28*28, number 6; channel :1
"""
CONV1_FILTER_SIZE = 5
CONV1_FILTER_CHANNEL = 1
CONV1_FILTER_NUM = 6

"""
第三层为卷积层, 卷积层_3的参数
input: filter:
       size 5 * 5   number 16 channel: 1
output: feature map
       size 10 * 10 number 16  channel: 1 
       
"""
CONV3_FILTER_SIZE = 5
CONV3_FILTER_CHANNEL = 1
CONV3_FILTER_NUM = 16

"""
第五层是一个卷积层, 
input: filter:
        size 5 *5    number 120   channel: 1
output: feature map
        size 1    number 120  channel: 1
"""
CONV5_FILTER_SIZE = 5
CONV5_FILTER_CHANNEL = 1
CONV5_FILTER_NUM = 120

# LeNet inference

In [121]:
    
"""conv 1.  第一层为卷积层,有6个filter
input size: 32 * 32 * 1
output size: 28 * 28 * 6
"""
conv1_filter = tf.Variable(tf.truncated_normal([CONV1_FILTER_SIZE,CONV1_FILTER_SIZE,CONV1_FILTER_CHANNEL, CONV1_FILTER_NUM],dtype=tf.float32))
conv1 = tf.nn.conv2d(inTensor, conv1_filter, strides=[1, 1, 1, 1], padding='VALID')
conv1_bias = tf.Variable(tf.truncated_normal([CONV1_FILTER_NUM], dtype=tf.float32))
relu_1 = tf.nn.relu(tf.nn.bias_add(conv1, conv1_bias))

"""pooling 2.  
 第2层为pooling层,与原始LetNet不同.原始的LeNet的pooling层有两个可训练参数
 input size: 28 * 28 * 6
 output size: 14 * 14 * 6
"""
pooling2 = tf.nn.avg_pool(relu_1, ksize=[1,2,2,1], strides=[1, 2, 2 ,1], padding="VALID")
relu_2 = tf.nn.relu(pooling2)


""" conv 3  
第3层为卷积层,共16个filter. 与原始LeNet不同,原始的LeNet是部分连接
这里为了方便是,整体连接,详细情况见论文希捷
input size 14 * 14 * 6
out putsize 10 * 10 *16
"""
conv3_filter = tf.Variable(tf.truncated_normal([5, 5, 6, 16]))
conv3_bias = tf.Variable(tf.truncated_normal([16]))
conv3 = tf.nn.conv2d(relu_2, conv3_filter, strides=[1,1,1,1],padding='VALID')
relu_3 =tf.nn.relu(tf.nn.bias_add(conv3, conv3_bias))


"""pooling 4  
第4层为pooling层, 与pooling 2 相同
input size:  10 * 10 * 16
output size: 5 * 5 * 16
"""
pooling4 = tf.nn.avg_pool(relu_3, ksize=[1,2,2,1],strides=[1,2,2,1],padding="VALID")
relu_4 = tf.nn.relu(pooling4)

"""conv 5
第5层为conv层, 这一层与原始的LeNet-5是一致的
input size: 5 * 5 * 16
output size: 1 * 1 * 120
"""
conv5_filter = tf.Variable(tf.truncated_normal([5, 5, 16, 120]))
conv5_bias = tf.Variable(tf.truncated_normal([120]))
conv5 = tf.nn.conv2d(relu_4, conv5_filter, strides=[1,1,1,1],padding="VALID")
relu_5 = tf.nn.relu(tf.nn.bias_add(conv5, conv5_bias))

"""fc 6
第6层为fc层,与原始不太一致,因为我只在mnist上进性试验
input size: BATCH_SIZE * 1 * 1 * 120
output size: BATCH_SIZE * 1 * 1 * 10
"""
fc_weights = tf.Variable(tf.truncated_normal([120,10]))
fc_bias = tf.Variable(tf.truncated_normal([10]))
relu_5 = tf.reshape(relu_5, [BATCH_SIZE, 120])
relu_6 = tf.nn.sigmoid(tf.nn.bias_add(tf.matmul(relu_5, fc_weights),fc_bias))


"""最后的输出单元
softmax分类器
"""
logits = tf.nn.softmax(relu_6)

In [122]:
batchLabels.shape

(100, 10)

In [123]:
# loss function
cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=batchLabels, logits=relu_6 )
loss = tf.reduce_mean(cross_entropy)

#train
train_op = tf.train.GradientDescentOptimizer(0.05).minimize(loss)


In [126]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(loss))
    print(sess.run(conv1_filter[:,:,0,0]))
    for i in range(1000):
        sess.run(train_op)
    print(sess.run(conv1_filter[:,:,0,0]))
    print(sess.run(loss))

2.44068
[[ 0.45515513  0.87927562 -1.86986887 -0.4473561  -1.53186631]
 [ 0.76428258  0.89881247 -0.45456028  0.3070229   0.59747416]
 [-1.41233289  1.31793773 -0.0540248  -1.08194458  0.36511591]
 [ 1.33356881  0.38394856 -0.6336869  -0.03608479  1.14866793]
 [-0.82556474  0.20747866 -0.29158038  1.24992657  0.05491471]]
[[ 0.45571333  0.87896752 -1.87000918 -0.44719428 -1.53188133]
 [ 0.76348394  0.89852756 -0.45353431  0.30837655  0.59813327]
 [-1.41404521  1.31715059 -0.05334353 -1.08123827  0.36528185]
 [ 1.33123732  0.38235626 -0.63454562 -0.03711052  1.14746463]
 [-0.82740611  0.20574446 -0.29353344  1.24770451  0.05291207]]
2.45421


In [115]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print(sess.run(logits))

[[ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
 [ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1

In [26]:
var = tf.Variable([[1,2,4]],dtype=tf.float32)

In [88]:
batch_samples, batch_labels=  mnist.train.next_batch(100)

In [91]:
print(batch_samples.shape)

(100, 784)


In [95]:
np.info(np.pad)

 pad(array, pad_width, mode, **kwargs)

Pads an array.

Parameters
----------
array : array_like of rank N
    Input array
pad_width : {sequence, array_like, int}
    Number of values padded to the edges of each axis.
    ((before_1, after_1), ... (before_N, after_N)) unique pad widths
    for each axis.
    ((before, after),) yields same before and after pad for each axis.
    (pad,) or int is a shortcut for before = after = pad width for all
    axes.
mode : str or function
    One of the following string values or a user supplied function.

    'constant'
        Pads with a constant value.
    'edge'
        Pads with the edge values of array.
    'linear_ramp'
        Pads with the linear ramp between end_value and the
        array edge value.
    'maximum'
        Pads with the maximum value of all or part of the
        vector along each axis.
    'mean'
        Pads with the mean value of all or part of the
        vector along each axis.
    'median'
        Pads with the med

array([[1, 2],
       [3, 4]])